# GraphSparqlQAChain

Graph databases are an excellent choice for applications based on network-like models. To standardize the syntax and semantics of such graphs, the W3C recommends Semantic Web Technologies, cp. [Semantic Web](https://www.w3.org/standards/semanticweb/). [SPARQL](https://www.w3.org/TR/sparql11-query/) serves as a query language analogously to SQL or Cypher for these graphs. This notebook demonstrates the application of LLMs as a natural language interface to a graph database by generating SPARQL.\
Disclaimer: To date, SPARQL query generation via LLMs is still a bit unstable. Be especially careful with UPDATE queries, which alter the graph.

There are several sources you can run queries against, including files on the web, files you have available locally, SPARQL endpoints, e.g., [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page), and [triple stores](https://www.w3.org/wiki/LargeTripleStores).

In [1]:
from langchain.chains import GraphSparqlQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import RdfGraph

## Load some RDF data

In [2]:
graph = RdfGraph(
    source_file="http://www.w3.org/People/Berners-Lee/card",
    standard="rdf",
    local_copy="test.ttl",
)

Note that providing a `local_file` is necessary for storing changes locally if the source is read-only.

## Refresh graph schema information
If the schema of the database changes, you can refresh the schema information needed to generate SPARQL queries.

In [3]:
graph.load_schema()

In [4]:
graph.get_schema

'In the following, each IRI is followed by the local name and optionally its label and its description in parentheses. \nThe RDF graph supports the following node types:\n<http://xmlns.com/foaf/0.1/PersonalProfileDocument> (PersonalProfileDocument, None, None), <http://www.w3.org/ns/auth/cert#RSAPublicKey> (RSAPublicKey, None, None), <http://www.w3.org/2000/10/swap/pim/contact#Male> (Male, None, None), <http://xmlns.com/foaf/0.1/Person> (Person, None, None), <http://www.w3.org/2006/vcard/ns#Work> (Work, None, None)\nThe RDF graph supports the following relationships:\n<http://xmlns.com/foaf/0.1/name> (name, None, None), <http://xmlns.com/foaf/0.1/based_near> (based_near, None, None), <http://www.w3.org/2000/01/rdf-schema#seeAlso> (seeAlso, None, None), <http://www.w3.org/ns/pim/space#storage> (storage, None, None), <http://usefulinc.com/ns/doap#developer> (developer, None, None), <http://xmlns.com/foaf/0.1/nick> (nick, None, None), <http://xmlns.com/foaf/0.1/title> (title, None, None),

## Querying the graph

Now, you can use the graph SPARQL QA chain to ask questions about the graph.

In [5]:
chain = GraphSparqlQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [6]:
chain.run("What is Timothy Berners-Lee's work homepage?")



> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?workHomepage
WHERE {
    ?person foaf:name "Timothy Berners-Lee" .
    ?person foaf:workplaceHomepage ?workHomepage .
}
Full Context:
[(rdflib.term.URIRef('https://www.w3.org/'),)]

> Finished chain.


"Timothy Berners-Lee's work homepage is https://www.w3.org/."

## Updating the graph

Analogously, you can update the graph, i.e., insert triples, using natural language.

In [7]:
chain.run(
    "Save that the person with the name 'Timothy Berners-Lee' has a work homepage at 'http://www.w3.org/foo/bar/'"
)



> Entering new GraphSparqlQAChain chain...
Identified intent:
UPDATE
Generated SPARQL:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
INSERT {
    ?person foaf:workplaceHomepage <http://www.w3.org/foo/bar/> .
}
WHERE {
    ?person foaf:name "Timothy Berners-Lee" .
}

> Finished chain.


'Successfully inserted triples into the graph.'

Let's verify the results:

In [8]:
query = """\
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?hp
WHERE {
    ?person foaf:name "Timothy Berners-Lee" . 
    ?person foaf:workplaceHomepage ?hp . 
}
"""
graph.query(query)

[(rdflib.term.URIRef('https://www.w3.org/'),),
 (rdflib.term.URIRef('http://www.w3.org/foo/bar/'),)]

## Now for an OWL ontology
Let's use data from a local file `graph_sparql_qa_local_input.ttl`.
The TBox provides notions for describing an actor, including their age.
The ABox includes a node for Tom Cruise and a triple stating his age to be 40.
Note that this deviates from what a vanilla LLM would return based on its training data.

In [9]:
graph = RdfGraph(
    source_file="graph_sparql_qa_local_input.ttl",
    standard="owl",
    language="en",
)

In [10]:
graph.load_schema()

In [11]:
graph.get_schema

'In the following, each IRI is followed by the local name and optionally its label and its description in parentheses. \nThe OWL graph supports the following node types:\n<http://example.org/example/Actor> (Actor, actor, An actor or actress is a person who acts in a dramatic production and who works in film, television, theatre, or radio in that capacity.), <http://example.org/example/AdministrativeRegion> (AdministrativeRegion, administrative region, A PopulatedPlace under the jurisdiction of an administrative body. This body may administer either a whole region or one or more adjacent Settlements (town administration)), <http://example.org/example/Animal> (Animal, animal, Kingdom of multicellular eukaryotic organisms.), <http://example.org/example/Person> (Person, person, Being that has certain capacities or attributes constituting personhood.), <http://example.org/example/Place> (Place, place, A small area known by a geographical name.)\nThe OWL graph supports the following object p

In [12]:
chain = GraphSparqlQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [13]:
chain.run("How old is Tom Cruise?")



> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX example: <http://example.org/example/>
SELECT ?age
WHERE {
    ?person example:name "Tom Cruise" .
    ?person example:dp1 ?age .
}
Full Context:
[(rdflib.term.Literal('40', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)]

> Finished chain.


'Tom Cruise is 40 years old.'

## Customization

The GraphSparqlQAChain relies on four prompts:
* `SPARQL_INTENT_PROMPT`: Identify the intent of the query, i.e., whether it is a SELECT or an UPDATE query 
* `SPARQL_GENERATION_SELECT_PROMPT`: Generate a SPARQL SELECT query
* `SPARQL_GENERATION_UPDATE_PROMPT`: Generate a SPARQL UPDATE query
* `SPARQL_QA_PROMPT`: Generate a natural language answer from the query results

The default prompts were designed using OpenAI models, especially gpt-3.5-turbo and newer, but they have also been shown to work well with other LLMs, such as StarCoder and GPT Neo. \
However, other models may work better using slightly adapted prompts.
You can override the default prompts as follows.

For instance, this is the default `SPARQL_GENERATION_SELECT_TEMPLATE`:

In [14]:
from langchain.chains.graph_qa.prompts import SPARQL_GENERATION_SELECT_TEMPLATE


SPARQL_GENERATION_SELECT_TEMPLATE

'Task: Generate a SPARQL SELECT statement for querying a graph database.\nFor instance, to find all email addresses of John Doe, the following query in backticks would be suitable:\n```\nPREFIX foaf: <http://xmlns.com/foaf/0.1/>\nSELECT ?email\nWHERE {{\n    ?person foaf:name "John Doe" .\n    ?person foaf:mbox ?email .\n}}\n```\nInstructions:\nUse only the node types and properties provided in the schema.\nDo not use any node types and properties that are not explicitly provided.\nInclude all necessary prefixes.\nSchema:\n{schema}\nNote: Be as concise as possible.\nDo not include any explanations or apologies in your responses.\nDo not respond to any questions that ask for anything else than for you to construct a SPARQL query.\nDo not include any text except the SPARQL query generated.\n\nThe question is:\n{prompt}'

It may be beneficial to adapt the SPARQL example to the ontology of interest.
You can define a custom prompt template and pass it to the GraphSparqlQAChain upon initialization like so:

In [15]:
from langchain.prompts.prompt import PromptTemplate


CUSTOM_SPARQL_GENERATION_SELECT_TEMPLATE = """Task: Generate a SPARQL SELECT statement for querying a graph database.
For instance, to find all people who are at least 30 years old and have a friend called "Sam Altman", the following query in backticks would be suitable:
```
PREFIX custom: <http://example.org/custom/>
SELECT ?person
WHERE {{
    ?person custom:age ?age .
    ?person custom:friend ?friend .
    ?friend custom:name "Sam Altman" .
    FILTER (?age >= 30)
}}
```
Instructions:
Use only the node types and properties provided in the schema.
Do not use any node types and properties that are not explicitly provided.
Include all necessary prefixes.
Schema:
{schema}
Note: Be as concise as possible.
Do not include any explanations or apologies in your responses.
Do not respond to any questions that ask for anything else than for you to construct a SPARQL query.
Do not include any text except the SPARQL query generated.

The question is:
{prompt}"""
CUSTOM_SPARQL_GENERATION_SELECT_PROMPT = PromptTemplate(
    input_variables=["schema", "prompt"], template=SPARQL_GENERATION_SELECT_TEMPLATE
)

CUSTOM_SPARQL_GENERATION_SELECT_PROMPT

PromptTemplate(input_variables=['prompt', 'schema'], template='Task: Generate a SPARQL SELECT statement for querying a graph database.\nFor instance, to find all email addresses of John Doe, the following query in backticks would be suitable:\n```\nPREFIX foaf: <http://xmlns.com/foaf/0.1/>\nSELECT ?email\nWHERE {{\n    ?person foaf:name "John Doe" .\n    ?person foaf:mbox ?email .\n}}\n```\nInstructions:\nUse only the node types and properties provided in the schema.\nDo not use any node types and properties that are not explicitly provided.\nInclude all necessary prefixes.\nSchema:\n{schema}\nNote: Be as concise as possible.\nDo not include any explanations or apologies in your responses.\nDo not respond to any questions that ask for anything else than for you to construct a SPARQL query.\nDo not include any text except the SPARQL query generated.\n\nThe question is:\n{prompt}')

In [16]:
custom_chain = GraphSparqlQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=graph,
    verbose=True,
    sparql_select_prompt=CUSTOM_SPARQL_GENERATION_SELECT_PROMPT
)